### 1. Install library

##### 1.1. Install library

In [ ]:
!pip install 'pymongo[srv]'

### 2. Connect to MongoDB

##### 2.1. Client connection

In [ ]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

uri = ""

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

##### 2.2. Create collection

In [ ]:
db = client['dsde']
books_collection = db['books']

### 3. Book model class

In [ ]:
import uuid
from typing import Optional
from pydantic import BaseModel, Field


class Book(BaseModel):
    id: str = Field(default_factory=lambda: str(uuid.uuid4()), alias="_id")
    title: Optional[str] = None
    author: Optional[str] = None
    synopsis: Optional[str] = None

class BookUpdate(BaseModel):
    title: Optional[str] = None
    author: Optional[str] = None
    synopsis: Optional[str] = None

### 4. Create book

In [ ]:
def create_book(book: Book):
    book_data = book.dict(by_alias=True)
    result = books_collection.insert_one(book_data)
    return str(result.inserted_id)

In [ ]:
new_book = Book(title="Don Quixote",
                author="Miguel de Cervantes",
                synopsis="A novel about a man who reads too many chivalric romances...")
book_id = create_book(new_book)
print(f"Inserted book with ID: {book_id}")

In [ ]:
# Schema free example

### 5. Read a book

In [ ]:
def get_book_by_id(book_id: str):
    book_data = books_collection.find_one({"_id": book_id})
    if book_data:
        return Book(**book_data)
    return None

In [ ]:
book = get_book_by_id(book_id)
print(book)

### 6. Update a book

In [ ]:
def update_book(book_id: str, book_update: BookUpdate):
    # To avoid setting key to null value
    update_data = {k: v for k, v in book_update.dict(
        exclude_unset=True).items() if v is not None}
    result = books_collection.update_one(
        {"_id": book_id}, {"$set": update_data})
    return result.modified_count

In [ ]:
book_update = BookUpdate(synopsis="Updated synopsis of Don Quixote")
updated_count = update_book(book_id, book_update)
print(f"Number of documents updated: {updated_count}")

In [ ]:
book = get_book_by_id(book_id)
print(book.synopsis)

### 7. Delete a book

In [ ]:
def delete_book(book_id: str):
    result = books_collection.delete_one({"_id": book_id})
    return result.deleted_count

In [ ]:
deleted_count = delete_book(book_id)
print(f"Number of documents deleted: {deleted_count}")

In [ ]:
book = get_book_by_id(book_id)

### 8. Input/ Output Formatting

In [ ]:
!wget -O book_list.json 'https://www.dropbox.com/scl/fi/6v797lfd22b371yp59l6m/book_list.json?rlkey=qf6wkxd03zubls8z7gsn35ppo&st=nrti7303&dl=0'

#### 8.1. Json to MongoDB

In [ ]:
import json

with open('/content/book_list.json', 'r') as file:
    books = json.load(file)

result = books_collection.insert_many(books)
print(f"Inserted {len(result.inserted_ids)} books into MongoDB.")

Check the inserted results

In [ ]:
book_list = list(books_collection.find())

print(book_list)

### 8.2 Result to DataFrame

In [ ]:
import pandas as pd

df = pd.DataFrame(book_list)
df